## 1. Preparations

In [ ]:
# to install pytorch on colab
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision


In [ ]:
!pip install -U bcolz

In [ ]:
!pip install Pillow==4.0.0

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import torchvision
from torchvision import models,transforms,datasets
import bcolz
import time
%matplotlib inline

We will first precompute the outputs of Vgg16 model on our dataset and store these values.

In [ ]:
use_gpu = torch.cuda.is_available()
print('Using gpu: %s ' % use_gpu)

dtype = torch.FloatTensor
if use_gpu:
    dtype = torch.cuda.FloatTensor

The following commands will download the dataset and need to be run only once.

In [ ]:
%mkdir data

In [ ]:
%cd data/
!wget http://files.fast.ai/data/dogscats.zip

In [ ]:
!unzip dogscats.zip

In [ ]:
def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()
def load_array(fname):
    return bcolz.open(fname)[:]

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

prep1 = transforms.Compose([
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                normalize,
            ])

In [ ]:
#uncomment the appropriate one
#laptop data_dir = '/home/lelarge/courses/data/dogscats'
#aws data_dir = '/home/ubuntu/data/dogscats'
#colab  d
data_dir = '/content/data/dogscats'

Initialize paths for dataset items

In [ ]:
dsets = {x: datasets.ImageFolder(os.path.join(data_dir, x), prep1)
         for x in ['train', 'valid']}

If you are running on CPU, you will probably need to lower the size of the batches. On Colab too :(

In [ ]:
batch_size = 4
#batch_size = 64

Initialize data loader that will fetch images from disk using num_workers parallel threads.

In [ ]:
dset_loaders = {x: torch.utils.data.DataLoader(dsets[x], batch_size=batch_size,
                                               shuffle=False, num_workers=0)
                for x in ['train', 'valid']}

In [ ]:
dset_sizes = {x: len(dsets[x]) for x in ['train', 'valid']}
dset_sizes

Instantiate VGG16 model pretrained on ImageNet from the ```torchvision``` model Zoo.

In [ ]:
model_vgg = models.vgg16(pretrained=True)

In [ ]:
if use_gpu:
    model_vgg = model_vgg.cuda()

By default all the modules are initialized to train mode (```self.training = True```). Also be aware that some layers have different behavior during train/and evaluation (like _BatchNorm_, _Dropout_) so setting it matters.

Also as a rule of thumb for programming in general, try to explicitly state your intent and set ```model.train()``` and ```model.eval()``` when necessary.

In [ ]:
model_vgg.eval()

## 2. Feature extraction

Function for extracting and storing CNN features, i.e. the ouput of VGG16 model in this case.

In [ ]:
def prefeat(dataset):
    features = []
    labels_list = []
    for data in dataset:
        inputs,labels = data
        if use_gpu:
            inputs , labels = inputs.cuda(),labels.cuda()
        else:
            inputs , labels = inputs,labels
        
        x = model_vgg(inputs)
        features.extend(x.data.cpu().numpy())
        labels_list.extend(labels.data.cpu().numpy())
    features = np.concatenate([[feat] for feat in features])
    return (features,labels_list)

In [ ]:
%%time
feat_train,lbs_train = prefeat(dset_loaders['train'])

In [ ]:
feat_train.shape

Loading and resizing the images every time we want to use them isn't necessary - instead we should save the processed arrays. By far the fastest way to save and load numpy arrays is using bcolz. This also compresses the arrays, so we save disk space. Here are the functions we'll use to save and load using bcolz (already loaded above...).

In [ ]:
%cd data/dogscats/

In [ ]:
%mkdir vgg16

In [ ]:
save_array(os.path.join(data_dir,'vgg16','feat_train.bc'),feat_train)
save_array(os.path.join(data_dir,'vgg16','lbs_train.bc'),lbs_train)

In [ ]:
%%time
feat_val,lbs_val = prefeat(dset_loaders['valid'])

In [ ]:
feat_val.shape

In [ ]:
save_array(os.path.join(data_dir,'vgg16','feat_val.bc'),feat_val)
save_array(os.path.join(data_dir,'vgg16','lbs_val.bc'),lbs_val)

In [ ]:
%cd /content/data/dogscats/
!zip -r vgg16 vgg16/*

In [ ]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
upload = drive.CreateFile({'title': 'vgg16_drive.zip'})
upload.SetContentFile('vgg16.zip')
upload.Upload()